In [45]:
from pyspark.sql import SparkSession
import pandas as pd

In [46]:
file = open('database.txt')

lines = file.readlines()
url = lines[0].rstrip()
driver = lines[1].rstrip()
user = lines[2].rstrip()
password = lines[3].rstrip()

In [47]:
spark = SparkSession.builder.appName('CreditCardSystem').getOrCreate()

In [89]:
brnch_df = spark.read.format('jdbc').options(
    url=url,
    driver=driver,
    dbtable='CDW_SAPP_BRANCH',
    user=user,
    password=password).load()

crdt_df = spark.read.format('jdbc').options(
    url=url,
    driver=driver,
    dbtable='CDW_SAPP_CREDIT_CARD',
    user=user,
    password=password).load()

cstmr_df = spark.read.format('jdbc').options(
    url=url,
    driver=driver,
    dbtable='CDW_SAPP_CUSTOMER',
    user=user,
    password=password).load()

In [90]:
brnch_df = brnch_df.toPandas()
crdt_df = crdt_df.toPandas()
cstmr_df = cstmr_df.toPandas()

In [50]:
dtled_cstmr = pd.merge(cstmr_df, crdt_df, on='CREDIT_CARD_NO', how='inner')
dtled_brnch = pd.merge(brnch_df, crdt_df, on='BRANCH_CODE', how='inner')

In [51]:
dtled_cstmr.head(5)

,CREDIT_CARD_NO,CUST_CITY,CUST_COUNTRY,CUST_EMAIL,CUST_PHONE,CUST_STATE,CUST_ZIP,FIRST_NAME,LAST_NAME,LAST_UPDATED,MIDDLE_NAME,SSN,FULL_STREET_ADDRESS,BRANCH_CODE,CUST_SSN,TRANSACTION_ID,TRANSACTION_TYPE,TRANSACTION_VALUE,TIMEID
0,4210653310061055,Natchez,United States,AHooper@example.com,123-7818,MS,39120,Alec,Hooper,2018-04-21 09:49:02,Wm,123456100,"Main Street North, 656",59,123456100,20528,Entertainment,46.60,2018-12-05
1,4210653310061055,Natchez,United States,AHooper@example.com,123-7818,MS,39120,Alec,Hooper,2018-04-21 09:49:02,Wm,123456100,"Main Street North, 656",145,123456100,20529,Gas,80.30,2018-07-01
2,4210653310061055,Natchez,United States,AHooper@example.com,123-7818,MS,39120,Alec,Hooper,2018-04-21 09:49:02,Wm,123456100,"Main Street North, 656",58,123456100,20530,Healthcare,27.69,2018-05-17
3,4210653310061055,Natchez,United States,AHooper@example.com,123-7818,MS,39120,Alec,Hooper,2018-04-21 09:49:02,Wm,123456100,"Main Street North, 656",93,123456100,20531,Grocery,42.57,2018-06-07
4,4210653310061055,Natchez,United States,AHooper@example.com,123-7818,MS,39120,Alec,Hooper,2018-04-21 09:49:02,Wm,123456100,"Main Street North, 656",8,123456100,20532,Test,26.96,2018-06-11


In [52]:
dtled_brnch.head(5)

,BRANCH_CITY,BRANCH_CODE,BRANCH_NAME,BRANCH_PHONE,BRANCH_STATE,BRANCH_STREET,BRANCH_ZIP,LAST_UPDATED,CREDIT_CARD_NO,CUST_SSN,TRANSACTION_ID,TRANSACTION_TYPE,TRANSACTION_VALUE,TIMEID
0,Lakeville,1,Example Bank,(123)456-5276,MN,Bridle Court,55044,2018-04-18 13:51:47,4210653349028689,123459988,77,Bills,53.03,2018-11-28
1,Lakeville,1,Example Bank,(123)456-5276,MN,Bridle Court,55044,2018-04-18 13:51:47,4210653312528499,123459918,239,Education,58.78,2018-11-26
2,Lakeville,1,Example Bank,(123)456-5276,MN,Bridle Court,55044,2018-04-18 13:51:47,4210653312528499,123459918,246,Bills,5.49,2018-04-04
3,Lakeville,1,Example Bank,(123)456-5276,MN,Bridle Court,55044,2018-04-18 13:51:47,4210653352364608,123451215,45319,Bills,1.68,2018-08-02
4,Lakeville,1,Example Bank,(123)456-5276,MN,Bridle Court,55044,2018-04-18 13:51:47,4210653362677974,123451185,45423,Bills,2.12,2018-05-12


2.1 Transaction Details Module

In [53]:
def zipTransactions(z, m, y):
    df = dtled_cstmr[(dtled_cstmr['CUST_ZIP'] == z) & (dtled_cstmr['TIMEID'].dt.month == m) & (dtled_cstmr['TIMEID'].dt.year == y)]
    df = df.sort_values(by = 'TIMEID', ascending = False)

    df = df[['FIRST_NAME', 'LAST_NAME', 'CUST_ZIP', 'TRANSACTION_VALUE', 'TIMEID']]

    return df

In [54]:
def typeTransaction(tType):
    return crdt_df[crdt_df['TRANSACTION_TYPE'] == tType]['TRANSACTION_VALUE'].sum()

In [55]:
def stateTransaction(state):
    fltr = dtled_brnch[dtled_brnch['BRANCH_STATE'] == state].groupby('BRANCH_CODE')['TRANSACTION_VALUE'].sum()
    fltr = fltr.to_frame()
    fltr = fltr.reset_index()

    fltr.columns = ['BRANCH ID', 'TOTAL TRANSACTIONS']
    
    return fltr

2.2 Customer Details Module

In [56]:
def checkAccount(cardId):
    return cstmr_df[cstmr_df['CREDIT_CARD_NO'] == cardId].transpose()

In [94]:
def updateAccount(cardId, name):
    global cstmr_df
    df = cstmr_df
    
    df.loc[cstmr_df['CREDIT_CARD_NO'] == cardId, 'FIRST_NAME'] == name
    df = spark.createDataFrame(df)

    df.write.format('jdbc').options(
        url='jdbc:mysql://localhost:3306/creditcard_capstone?permitMysqlScheme',
        driver='org.mariadb.jdbc.Driver',
        dbtable='cdw_sapp_customer',
        user='root',
        password='123456').mode('append').save()

In [61]:
def createBill(cardId, m, yr):
    df = dtled_cstmr[(dtled_cstmr['CREDIT_CARD_NO'] == cardId) & (dtled_cstmr['TIMEID'].dt.month == m) & (dtled_cstmr['TIMEID'].dt.year == yr)]['TRANSACTION_VALUE'].sum()
    
    return df.round(2)

In [62]:
def transactionHistory(cardId, start, end):
    return dtled_cstmr[(dtled_cstmr['CREDIT_CARD_NO'] == cardId) & (dtled_cstmr['TIMEID'].between(start, end))][[ 'TIMEID', 'TRANSACTION_TYPE', 'TRANSACTION_VALUE']]

Console Program Starts Here

In [95]:
print('Welcome to Perscholas Banking Terminal')

print('')

print('What would you like to do? Type 1 for Transaction Details. Type 2 for Customer Details')

choice = int(input('Choose Your Options (1 or 2): '))

if choice == 1:
    print('You have chosen: Transaction Details')

    print('')

    print('Choose the Following Options: Type 1 for Customer Transaction Details by Zip Code. Type 2 for Transaction Details by Type. Type 3 for Branch Transaction Details by State')

    choice2 = int(input('Choose Your Options (1, 2 or 3): '))

    if choice2 == 1:
        print('You have chosen: Customer Transaction Details by Zip Code.')

        print('')

        print('Please Enter the Information in the Following Order: Zip Code, Month and Year.')
        
        zpcd = int(input('Desired Zip Code: '))
        m = int(input('Desired Month: '))
        y = int(input('Desired Year: '))

        print('Inputs Recieved: ' + str(zpcd) + ' ' + str(m) + ' ' + str(y))

        print('')

        print('Generating Results')

        print('Transactions in the Zip Code: ' + str(zpcd) + ', the Month: ' + str(m) + ', and the Year: ' + str(y) + ', in Decending Order: ')

        print('')
        
        print(zipTransactions(zpcd, m, y))

    elif choice2 == 2:
        print('You have chosen: Transaction Details by Type')

        print('')

        print('Please Enter What Type of Transaction You Want to See: ')

        typ = input('Desired Type: ')

        print('Type Chosen: ' + '"' + typ + '"')

        print('')

        print('Generating Results')

        print('Total Transactions for ' + '"' + typ + '"'+ ' is: '+ str(typeTransaction(typ)))

    elif choice2 == 3:
        print('You have chosen: Branch Transaction Details by State')
        
        print('')

        print('Please Enter What State You Want: ')

        state = input('Desired State: ')

        print('State Chosen: ' + '"' + state + '"')

        print('')

        print('Generating Results')

        print('Total Transactions in ' + '"' + state + '"'+ ' by Branches: ')

        print('')

        print(stateTransaction(state))
        
    else:
        print('Try Again')

elif choice == 2:
    print('You have chosen: Customer Details')
    
    print('')

    print('Choose the Following Options: Type 1 to Check Account Details. Type 2 to Update Account Details. Type 3 for Create Monthly Bill. Type 4 to See Transaction History')

    choice3 = int(input('Choose Your Options (1, 2, 3 or 4): '))

    if choice3 == 1:
        print('You have chosen: Check Account Details')
        
        print('')
        
        print('Please Enter in Your Credit Card Number: ')

        cardId = input('Credit Card Number: ')

        print('')

        print('Getting Account Details')

        print('')

        print(checkAccount(cardId))

    elif choice3 == 2:
        print('You gave chosen: Update Account')

        print('')

        print('Please Enter Your Credit Card Number: ')
        cardId = input('Credit Card Number: ')
        
        print('')

        print('Change Your First Name')
        name = input('Your First Name: ')

        print('')

        print(updateAccount(cardId, name))

        print('Action Completed')

        print('')

        print('Have a Nice Day!')
    
    elif choice3 == 3:
        print('You have chosen: Check Monthly Bill')
        
        print('')
        
        print('Please Enter Your Credit Card Number: ')

        cardId = input('Credit Card Number: ')
        m = int(input('Desired Month: '))
        y = int(input('Desired Year: '))

        print('Generating Monthly Bill')

        print('')
        
        print('Amount Due: ')

        print(createBill(cardId, m, y))

        print('')

        print('Have a Nice Day!')
        
    elif choice3 == 4:
        print('You have chosen: Transaction History')
        
        print('')
        
        print('Please Enter Your Credit Card Number: ')

        cardId = input('Credit Card Number: ')

        print('')

        print('Please the Start and End Dates: ')

        s = input('Start Date (Year-Month-Day): ')
        e = input('End Date (Year-Month-Day): ')

        print('Generating Transaction History')

        print('')
    
        print(transactionHistory(cardId, s, e))

    else:
        print('Try Again')

else:
    print('Try Again')

Welcome to Perscholas Banking Terminal

What would you like to do? Type 1 for Transaction Details. Type 2 for Customer Details
You have chosen: Customer Details

Choose the Following Options: Type 1 to Check Account Details. Type 2 to Update Account Details. Type 3 for Create Monthly Bill. Type 4 to See Transaction History
You gave chosen: Update Account

Please Enter Your Credit Card Number: 

Change Your First Name



Py4JJavaError: An error occurred while calling o306.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 17.0 failed 1 times, most recent failure: Lost task 10.0 in stage 17.0 (TID 49) (DESKTOP-6JUSJFI executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoin

In [76]:
cstmr_df[cstmr_df['CREDIT_CARD_NO'] == '4210653310195948']

,CREDIT_CARD_NO,CUST_CITY,CUST_COUNTRY,CUST_EMAIL,CUST_PHONE,CUST_STATE,CUST_ZIP,FIRST_NAME,LAST_NAME,LAST_UPDATED,MIDDLE_NAME,SSN,FULL_STREET_ADDRESS
3,4210653310195948,NewBerlin,United States,EHardy@example.com,124-3215,WI,53151,Eugenio,Hardy,2018-04-21 09:49:02,Trina,123459758,"Country Club Road, 253"
955,4210653310195948,NewBerlin,United States,EHardy@example.com,124-3215,WI,53151,Eugenio,Hardy,2018-04-21 09:49:02,Trina,123459758,"Country Club Road, 253"
1907,4210653310195948,NewBerlin,United States,EHardy@example.com,124-3215,WI,53151,Eugenio,Hardy,2018-04-21 09:49:02,Trina,123459758,"Country Club Road, 253"
